### <center> Algs and Data Structures Assignment 3 <center>
##### <center> Jon Martins <center>

### Question 1

In [1]:
def string_match(String1, String2): # The function is defined to return the common prefix between the two strings.
    def common_prefix(s1, s2): 
        prefix = [] # List is created to store matching characters.
        for c1, c2 in zip(s1, s2): # Characters from both strings are iterated at the same time.
            if c1 == c2:
                prefix.append(c1) # Matching characters are added to the prefix list.
            else:
                break # The loop stops if there is a mismatch.
        return "".join(prefix) # The list is converted to a string and then returned.

    n = len(String1) # The length of String1 is retrieved and stored as n.
    max_prefix = "" # The longest common prefix  found is stored.
    min_shifts = -1 # The number of shifts that gives the max prefix is stored.

    for shift in range(n): # Every possible rotation is tested of String2
        rotated_string = String2[shift:] + String2[:shift] # String 2 is rotated.
        current_prefix = common_prefix(rotated_string, String1) # The common prefix between the rotated String2 and String1 is calculated.

        if len(current_prefix) > len(max_prefix): # The if statement updates the longest prefix length if the current prefix length is longer.
            max_prefix = current_prefix
            min_shifts = shift

    return min_shifts, max_prefix # The minimum shifts and maximum prefix is returned

String1 = "Bioinformatics"
String2 = "This is Bioinformatics"
shifts, prefix = string_match(String1, String2)
print(f"Shift: {shifts}")
print(f"Prefix: {prefix}")

Shift: 8
Prefix: Bioinformatics


### Question 2

In [2]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=0, gap=-1): # The needleman-wunch function is defined in order to get matches,mismatches, and gaps between sequence 1 and 2.
    
    m, n = len(seq1), len(seq2) # The lengths of each sequence is assigned to m and n respectively.
    
    score_matrix = [[0] * (n + 1) for _ in range(m + 1)] # A matrix is created as (m+1) x (n+1), initialized at 0.
    
    for i in range(m + 1): # the first row and column are initialized with gap penalties.
        score_matrix[i][0] = gap * i
    for j in range(n + 1):
        score_matrix[0][j] = gap * j
    
    for i in range(1, m + 1): # The following for loop uses the match score if the characters match, oterwise it uses the mismatch score.
        for j in range(1, n + 1):
            if seq1[i - 1] == seq2[j - 1]:
                diag = score_matrix[i - 1][j - 1] + match
            else:
                diag = score_matrix[i - 1][j - 1] + mismatch
            up = score_matrix[i - 1][j] + gap # The score for inserting a gap in sequence 2 is calculated, when it is moving up.
            left = score_matrix[i][j - 1] + gap # the score for inserting a gap in sequence 1 is calculated, when moving left.
            score_matrix[i][j] = max(diag, up, left) # The highest score is then chosen and then put into the matrix.

    align1 = "" # Empty strings are created for aligned sequences.
    align2 = ""
    i, j = m, n # The traceback is started from the bottom-right corner of the matrix, where i and j are equal to m and n.
    
    while i > 0 or j > 0: # The while statement will reconstruct the alignment.
        current = score_matrix[i][j]
        if i > 0 and j > 0: # The if statement will compute the diagonal score if it is not on the matrix boundary.
            if seq1[i - 1] == seq2[j - 1]:
                score_diag = score_matrix[i - 1][j - 1] + match
            else:
                score_diag = score_matrix[i - 1][j - 1] + mismatch
        else:
            score_diag = float('-inf') # The diagonal score is set to negative infinity if the algorithm is at the edge of the matrix, no longer being able to move diagonal.
        
        score_up = score_matrix[i - 1][j] + gap if i > 0 else float('-inf') # The score is computed when the gap is moved up in sequence 2.
        score_left = score_matrix[i][j - 1] + gap if j > 0 else float('-inf') # The score is computed when the gap is moved left in sequence 1.

        if i > 0 and j > 0 and current == score_diag: # The direction in which the score is from is checked and the alignment is built from that.
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif i > 0 and current == score_up: # The gap in sequence 2 is moved up.
            align1 = seq1[i - 1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1 # The gap in sequence 1 is moved left.
            align2 = seq2[j - 1] + align2
            j -= 1

    return score_matrix, align1, align2 # The final score matrix, along with the two aligned sequenes, is returned.


seq1 = "GCAGG"
seq2 = "AGCTCAG"

# The algorithm is run.
matrix, aligned_seq1, aligned_seq2 = needleman_wunsch(seq1, seq2)

# The matrix and final aligned sequences are printed.
print("Score Matrix:")
for row in matrix:
    print(row)
print("\nAligned Sequences:")
print(aligned_seq1)
print(aligned_seq2)


Score Matrix:
[0, -1, -2, -3, -4, -5, -6, -7]
[-1, 0, 0, -1, -2, -3, -4, -5]
[-2, -1, 0, 1, 0, -1, -2, -3]
[-3, -1, -1, 0, 1, 0, 0, -1]
[-4, -2, 0, -1, 0, 1, 0, 1]
[-5, -3, -1, 0, -1, 0, 1, 1]

Aligned Sequences:
-GC-AGG
AGCTCAG


### Question 3

In [3]:
def nucleotide_to_num(nucleotide): # The nucleotide_to_num function converts each nucleotide to a number.
    return {'A': 0, 'C': 1, 'G': 2, 'T': 3}[nucleotide]

def rabin_karp(text, pattern, d, q): # The Rabin-Karp algorithm is defined with text being the full DNA sequence, pattern being the sequence search term, d being the size,, and q being the prime number being used.
    n = len(text)
    m = len(pattern)
    h = pow(d, m - 1, q) # The leftmost character in the rolling hash is removed.
    p = 0 # p is the hash for the pattern.
    t = 0 # t us the hash for the current window in the text.
    result = [] # the list of matching start indicies is shown.

    for i in range(m): # This for loop calculates the initial hash values for the pattern and the first sbstring of the text.
        p = (d * p + nucleotide_to_num(pattern[i])) % q
        t = (d * t + nucleotide_to_num(text[i])) % q

    for i in range(n - m + 1): # The following loop states that if the hash of the pattern matches the hash of the current text, add the string to the results.
        if p == t:
            if text[i:i + m] == pattern: # Thisif statement is to double check that there is a match.
                result.append(i)
        if i < n - m: # This if loop is used to remove the character going out and add the character that is going in.
            t = (d * (t - nucleotide_to_num(text[i]) * h) + nucleotide_to_num(text[i + m])) % q
            if t < 0:
                t += q

    return result # the result is returned.

text = "AGCATGCAGCAT"
pattern = "GCAT"
d = 4
q = 13
matches = rabin_karp(text, pattern, d, q)
print(matches)

[1, 8]
